In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import cdflib

In [2]:
base_f_loc = '/storage/silver/stfc_cg/hf832176/data/THEMIS/'

## Basic commands:

data = some .cdf file.

data.cdf_info() - lists variables and more

data.varattsget() - lists variables

data.varattsget(variable_name) - lists variable info

In [33]:
spacecraft = 'the'

In [34]:
def get_file_paths(base_f_loc, instrument, spacecraft):
    f_path = base_f_loc+instrument+'/'+spacecraft
    files = !ls {f_path}?*.cdf
    return files

In [35]:
# files[100]

In [36]:
# instrument = 'sst'
# files = get_file_paths(base_f_loc, instrument, spacecraft)
# data = cdflib.CDF('/storage/silver/stfc_cg/hf832176/data/THEMIS/sst/tha_l2_sst_20070926_v01.cdf')#files[100])
# # eflux = data.varget(spacecraft+'_psef_en_eflux')
# # data.varattsget()
# data.cdf_info()

In [37]:
# data.varget('tha_psef_data_quality')

In [38]:
# data.varattsget('thb_peef_en_eflux')

In [39]:
# data.varattsget('tha_psef_en_eflux')

# <center>SST data: Spectra</center>

In [40]:
instrument = 'sst'
files = get_file_paths(base_f_loc, instrument, spacecraft)

sub_dfs = []
for i in files:
#     print(i)
    # Load data
    try:
        data = cdflib.CDF(i)
    except OSError:
        print('File failed to load: '+i)
        pass
    # Read electron spectrogram
    try:
        eflux = data.varget(spacecraft+'_psef_en_eflux')
        # Read spectrogram quality flag (>0 is bad)
        quality = data.varget(spacecraft+'_psef_data_quality')
    except ValueError:
        print('Variable not present')
        continue
    
    # Read energy levels for data
    ene = data.varget(spacecraft+'_psef_en_eflux_yaxis')
    e_col_names = ['E_'+str(i+1) for i in range(len(ene[0]))]
    # Create datetimes for timestamps
    try:
        dti = [dt.datetime(1970,1,1) + dt.timedelta(seconds=i)
           for i in data.varget(spacecraft+'_psef_time')]
    except TypeError:
        print('No data: '+i)
    
    if len(quality) < len(eflux):
        quality2 = [np.nan]*len(eflux)
        for item, idx in enumerate(quality):
            quality2[idx] = item
        quality = quality2
    df = pd.DataFrame(eflux,index=dti,columns=e_col_names)
    df['quality'] = quality
    
    sub_dfs.append(df)

File failed to load: /storage/silver/stfc_cg/hf832176/data/THEMIS/sst/the_l2_sst_20070927_v01.cdf
File failed to load: /storage/silver/stfc_cg/hf832176/data/THEMIS/sst/the_l2_sst_20070928_v01.cdf
File failed to load: /storage/silver/stfc_cg/hf832176/data/THEMIS/sst/the_l2_sst_20070929_v01.cdf
File failed to load: /storage/silver/stfc_cg/hf832176/data/THEMIS/sst/the_l2_sst_20071005_v01.cdf


In [41]:
# instrument = 'sst'
# files = get_file_paths(base_f_loc, instrument, spacecraft)
# data = cdflib.CDF(files[0])
# data.varattsget(spacecraft+'_psef_en_eflux')

In [42]:
comb_df = pd.concat(sub_dfs)
comb_df.info()
comb_df.to_csv(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.csv')
comb_df.to_pickle(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.pkl')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4974832 entries, 2007-09-15 00:03:23.082396 to 2010-04-01 00:05:25.610651
Data columns (total 17 columns):
 #   Column   Dtype  
---  ------   -----  
 0   E_1      float64
 1   E_2      float64
 2   E_3      float64
 3   E_4      float64
 4   E_5      float64
 5   E_6      float64
 6   E_7      float64
 7   E_8      float64
 8   E_9      float64
 9   E_10     float64
 10  E_11     float64
 11  E_12     float64
 12  E_13     float64
 13  E_14     float64
 14  E_15     float64
 15  E_16     float64
 16  quality  int32  
dtypes: float64(16), int32(1)
memory usage: 664.2 MB


In [43]:
# instrument = 'sst'
# spacecraft = 'tha'
# files = get_file_paths(base_f_loc, instrument, spacecraft)
# data = cdflib.CDF(files[0])
# ene = data.varget(spacecraft+'_psef_en_eflux_yaxis')
# ene[0]
# df = pd.DataFrame([i for i in range(len(ene[0]))],index=ene[0],columns=['eV'])
# df.to_csv(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'_energy_levels.csv')
# df.to_pickle(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'_energy_levels.pkl')

# <center> STATE: Position, orientation etc. </center>

In [44]:
instrument = 'state'
files = get_file_paths(base_f_loc, instrument, spacecraft)

sub_dfs = []
for i in files:
    # Load data
    try:
        data = cdflib.CDF(i)
    except OSError:
        print('File failed to load: '+i)
        pass
    # Read GSM position
    pos_gsm = data.varget(spacecraft+'_pos_gsm')
    pos_gei = data.varget(spacecraft+'_pos')
    
    spin_ra = data.varget(spacecraft+'_spinras') # right ascension
    spin_dec = data.varget(spacecraft+'_spindec') # declination
    spin_per = data.varget(spacecraft+'_spinper') # spin period
    spin_phase = data.varget(spacecraft+'_spinphase') #spin phase
    
    # Create datetimes for timestamps
    try:
        dti = [dt.datetime(1970,1,1) + dt.timedelta(seconds=i)
           for i in data.varget(spacecraft+'_state_time')]
    except TypeError:
        print('No data: '+i)
    
    df = pd.DataFrame(pos_gsm,index=dti,columns=['pos_x_gsm','pos_y_gsm','pos_z_gsm'])
    df['spin_ra'] = spin_ra
    df['spin_dec'] = spin_dec
    df['spin_per'] = spin_per
    df['spin_phase'] = spin_phase
    df['pos_x_gei'] = pos_gei[:,0]
    df['pos_y_gei'] = pos_gei[:,1]
    df['pos_z_gei'] = pos_gei[:,2]
    
    sub_dfs.append(df)

In [45]:
# instrument = 'state'
# spacecraft = 'tha'
# files = get_file_paths(base_f_loc, instrument, spacecraft)

# data = cdflib.CDF(files[0])

# data.varattsget('tha_pos')

In [46]:
comb_df = pd.concat(sub_dfs)
comb_df.info()
comb_df.to_csv(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.csv')
comb_df.to_pickle(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.pkl')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1052640 entries, 2007-09-15 00:00:00 to 2010-03-31 23:59:00
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   pos_x_gsm   1052640 non-null  float32
 1   pos_y_gsm   1052640 non-null  float32
 2   pos_z_gsm   1052640 non-null  float32
 3   spin_ra     1052640 non-null  float32
 4   spin_dec    1052640 non-null  float32
 5   spin_per    1052640 non-null  float32
 6   spin_phase  1052640 non-null  float32
 7   pos_x_gei   1052640 non-null  float32
 8   pos_y_gei   1052640 non-null  float32
 9   pos_z_gei   1052640 non-null  float32
dtypes: float32(10)
memory usage: 48.2 MB


# <center> FGM: B-field etc. </center>

In [47]:
instrument = 'fgm'
data_rate = 'l' # or 'h'
files = get_file_paths(base_f_loc, instrument, spacecraft)

sub_dfs = []
for i in files:
    # Load data
    try:
        data = cdflib.CDF(i)
    except OSError:
        print('File failed to load: '+i)
        pass
    # Read GSM position
    try:
        b_gsm = data.varget(spacecraft+'_fg'+data_rate+'_gsm')
    except ValueError as err:
        print("Value error: {0}".format(err))
        print(i)
        pass
    except EOFError as err:
        print("EOF error: {0}".format(err))
        print(i)
        pass
    
    try:    
        quality = data.varget(spacecraft+'_fg'+data_rate+'_gsmQ')
    except ValueError as err:
        print("Value error: {0}".format(err))
        print(i)
        pass
    except EOFError as err:
        print("EOF error: {0}".format(err))
        print(i)
        pass
    
    # Create datetimes for timestamps
    try:
        dti = [dt.datetime(1970,1,1) + dt.timedelta(seconds=i)
               for i in data.varget(spacecraft+'_fg'+data_rate+'_time')]
    except TypeError:
        print('No data: '+i)
    except ValueError as err:
        print("Value error: {0}".format(err))
        print(i)
        pass
    except EOFError as err:
        print("EOF error: {0}".format(err))
        print(i)
        pass

    
    df = pd.DataFrame(b_gsm,index=dti,columns=['Bx_gsm','By_gsm','Bz_gsm'])
    df['quality'] = quality
    
    sub_dfs.append(df)

No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20070927_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20070928_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20070929_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20071005_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20081003_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20081208_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20090711_v01.cdf
No data: /storage/silver/stfc_cg/hf832176/data/THEMIS/fgm/the_l2_fgm_20090715_v01.cdf


In [48]:
comb_df = pd.concat(sub_dfs)
comb_df.info()
comb_df.to_csv(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.csv')
comb_df.to_pickle(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.pkl')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 138960896 entries, 2007-09-15 00:00:25.464584 to 2010-03-31 13:00:08.858704
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   Bx_gsm   float32
 1   By_gsm   float32
 2   Bz_gsm   float32
 3   quality  object 
dtypes: float32(3), object(1)
memory usage: 3.6+ GB


# <center> ESA Data: Spectra </center>

In [49]:
instrument = 'esa'
files = get_file_paths(base_f_loc, instrument, spacecraft)

sub_dfs = []
for i in files[:]:
    # Load data
    try:
        data = cdflib.CDF(i)
    except OSError:
        print('File failed to load: '+i)
        pass
    # Read electron spectrogram
    try:
        eflux = data.varget(spacecraft+'_peef_en_eflux')
        # Read spectrogram quality flag (>0 is bad)
        quality = data.varget(spacecraft+'_peef_data_quality')
        # Read energy levels for data
        ene = data.varget(spacecraft+'_peef_en_eflux_yaxis')
    except ValueError:
        print('Variable not found')
        continue
    
    try:
        e_col_names = ['E_'+str(i+1) for i in range(len(ene[0]))]
    except TypeError:
        print('No energy data: '+i)
    # Create datetimes for timestamps
    try:
        dti = [dt.datetime(1970,1,1) + dt.timedelta(seconds=i)
           for i in data.varget(spacecraft+'_peef_time')]
    except TypeError:
        print('No time data: '+i)
    
    df = pd.DataFrame(eflux,index=dti,columns=e_col_names)
    df['quality'] = quality
    
    sub_dfs.append(df)
    
#     display(df.head(2))
#     display(eflux)

No energy data: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070927_v01.cdf
No time data: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070927_v01.cdf
File failed to load: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070928_v01.cdf
No energy data: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070928_v01.cdf
No time data: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070928_v01.cdf
No energy data: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070929_v01.cdf
No time data: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20070929_v01.cdf
File failed to load: /storage/silver/stfc_cg/hf832176/data/THEMIS/esa/the_l2_esa_20071005_v01.cdf


In [50]:
comb_df = pd.concat(sub_dfs)
comb_df.info()
comb_df.to_csv(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.csv')
comb_df.to_pickle(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'.pkl')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4996745 entries, 2007-09-15 00:00:34.165417 to 2010-03-31 23:54:33.952057
Data columns (total 33 columns):
 #   Column   Dtype  
---  ------   -----  
 0   E_1      float64
 1   E_2      float64
 2   E_3      float64
 3   E_4      float64
 4   E_5      float64
 5   E_6      float64
 6   E_7      float64
 7   E_8      float64
 8   E_9      float64
 9   E_10     float64
 10  E_11     float64
 11  E_12     float64
 12  E_13     float64
 13  E_14     float64
 14  E_15     float64
 15  E_16     float64
 16  E_17     float64
 17  E_18     float64
 18  E_19     float64
 19  E_20     float64
 20  E_21     float64
 21  E_22     float64
 22  E_23     float64
 23  E_24     float64
 24  E_25     float64
 25  E_26     float64
 26  E_27     float64
 27  E_28     float64
 28  E_29     float64
 29  E_30     float64
 30  E_31     float64
 31  E_32     float64
 32  quality  object 
dtypes: float64(32), object(1)
memory usage: 1.3+ GB


In [51]:
# instrument = 'esa'
# spacecraft = 'tha'
# files = get_file_paths(base_f_loc, instrument, spacecraft)
# data = cdflib.CDF(files[20])
# ene = data.varget(spacecraft+'_peef_en_eflux_yaxis')
# ene[0]
# df = pd.DataFrame([i for i in range(len(ene[0]))],index=ene[0],columns=['eV'])
# df.to_csv(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'_energy_levels.csv')
# df.to_pickle(base_f_loc+spacecraft+'/'+spacecraft+'_'+instrument+'_energy_levels.pkl')